# Binary Classification Example

The Pipelines API provides higher-level API built on top of DataFrames for constructing ML pipelines.
You can read more about the Pipelines API in the [programming guide](https://spark.apache.org/docs/latest/ml-guide.html).

**Binary Classification** is the task of predicting a binary label.
E.g., is an email spam or not spam? Should I show this ad to this user or not? Will it rain tomorrow or not?
This section demonstrates algorithms for making these types of predictions.

## Dataset Review

The Adult dataset we are going to use is publicly available at the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Adult).
This data derives from census data, and consists of information about 48842 individuals and their annual income.
We will use this information to predict if an individual earns >50k a year or <=50K a year.
The dataset is rather clean, and consists of both numeric and categorical variables.

Attribute Information:

- age: continuous
- workclass: Private,Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked
- fnlwgt: continuous
- education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc...
- education-num: continuous
- marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent...
- occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners...
- relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried
- race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black
- sex: Female, Male
- capital-gain: continuous
- capital-loss: continuous
- hours-per-week: continuous
- native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany...

Target/Label: - <=50K, >50K

## Load Data

In this example, we will read in the Adult dataset from databricks-datasets.
We'll read in the data in SQL using the CSV data source for Spark and rename the columns appropriately.

In [7]:
%fs ls databricks-datasets/adult/adult.data

In [8]:
%sql DROP TABLE IF EXISTS adult

In [9]:
%sql
CREATE TABLE adult (
  age DOUBLE,
  workclass STRING,
  fnlwgt DOUBLE,
  education STRING,
  education_num DOUBLE,
  marital_status STRING,
  occupation STRING,
  relationship STRING,
  race STRING,
  sex STRING,
  capital_gain DOUBLE,
  capital_loss DOUBLE,
  hours_per_week DOUBLE,
  native_country STRING,
  income STRING)
USING com.databricks.spark.csv
OPTIONS (path "/databricks-datasets/adult/adult.data", header "true")


In [10]:
dataset = spark.table("adult")
cols = dataset.columns

In [11]:
display(dataset)

## Preprocess Data

Since we are going to try algorithms like Logistic Regression, we will have to convert the categorical variables in the dataset into numeric variables.
There are 2 ways we can do this.

* Category Indexing

  This is basically assigning a numeric value to each category from {0, 1, 2, ...numCategories-1}.
  This introduces an implicit ordering among your categories, and is more suitable for ordinal variables (eg: Poor: 0, Average: 1, Good: 2)

* One-Hot Encoding

  This converts categories into binary vectors with at most one nonzero value (eg: (Blue: [1, 0]), (Green: [0, 1]), (Red: [0, 0]))

In this dataset, we have ordinal variables like education (Preschool - Doctorate), and also nominal variables like relationship (Wife, Husband, Own-child, etc).
For simplicity's sake, we will use One-Hot Encoding to convert all categorical variables into binary vectors.
It is possible here to improve prediction accuracy by converting each categorical column with an appropriate method.

Here, we will use a combination of [StringIndexer] and [OneHotEncoderEstimator] to convert the categorical variables.
The `OneHotEncoderEstimator` will return a [SparseVector].

Since we will have more than 1 stages of feature transformations, we use a [Pipeline] to tie the stages together.
This simplifies our code.

[StringIndexer]: http://spark.apache.org/docs/latest/ml-features.html#stringindexer
[OneHotEncoderEstimator]: https://spark.apache.org/docs/latest/ml-features.html#onehotencoderestimator
[SparseVector]: https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.linalg.SparseVector
[Pipeline]: http://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.Pipeline

In [13]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
categoricalColumns = ["workclass", "education", "marital_status", "occupation", "relationship", "race", "sex", "native_country"]
stages = [] # stages in our Pipeline
for categoricalCol in categoricalColumns:
    # Category Indexing with StringIndexer
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    # Use OneHotEncoder to convert categorical variables into binary SparseVectors
    # encoder = OneHotEncoderEstimator(inputCol=categoricalCol + "Index", outputCol=categoricalCol + "classVec")
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    # Add stages.  These are not run here, but will run all at once later on.
    stages += [stringIndexer, encoder]

The above code basically indexes each categorical column using the `StringIndexer`,
and then converts the indexed categories into one-hot encoded variables.
The resulting output has the binary vectors appended to the end of each row.

We use the `StringIndexer` again to encode our labels to label indices.

In [15]:
# Convert label into label indices using the StringIndexer
label_stringIdx = StringIndexer(inputCol="income", outputCol="label")
stages += [label_stringIdx]

Next, we will use the `VectorAssembler` to combine all the feature columns into a single vector column.
This will include both the numeric columns and the one-hot encoded binary vector columns in our dataset.

In [17]:
# Transform all features into a vector using VectorAssembler
numericCols = ["age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week"]
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

We finally run our stages as a Pipeline.
This puts the data through all of the feature transformations we described in a single call.

In [19]:
# Create a Pipeline.
pipeline = Pipeline(stages=stages)
# Run the feature transformations.
#  - fit() computes feature statistics as needed.
#  - transform() actually transforms the features.
pipelineModel = pipeline.fit(dataset)
dataset = pipelineModel.transform(dataset)
# Keep relevant columns
selectedcols = ["label", "features"] + cols
dataset = dataset.select(selectedcols)
display(dataset)

In [20]:
### Randomly split data into training and test sets. set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed=100)
print(trainingData.count())
print(testData.count())

In [21]:
#This section shows how to use MLeap to load a trained model for use in your application. To use an existing ML models and pipelines to make predictions for new data, you can deserialize the model from the file you saved.

#Import Model to PySpark
#This section shows how to load an MLeap bundle and make predictions on a Spark DataFrame. This can be useful if you want to use the same persistence format (bundle) for loading into Spark and non-Spark applications. If your goal is to make predictions only in Spark, then we recommend using MLlib's native ML persistence.

In [22]:
display(dbutils.fs.ls("/FileStore/model"))

In [23]:
import mleap.pyspark
from mleap.pyspark.spark_support import SimpleSparkSerializer

from pyspark.ml import PipelineModel
deserializedPipeline = PipelineModel.deserializeFromBundle("jar:file:/dbfs/FileStore/model/binaryclassifier_pipeline-json.zip")

In [24]:
#Now you can use the loaded model to make predictions.
display(testData)

In [25]:
exampleResults = deserializedPipeline.transform(testData)
display(exampleResults)

In [26]:
exampleResults.printSchema()

In [27]:
num = exampleResults.count()
num

In [28]:
output = exampleResults.take(num)

In [29]:
output

In [30]:
exampleResults.dtypes

In [31]:
display(dbutils.fs.ls("/FileStore/model/"))

In [32]:
#Convert the RDD to dataframe
from pyspark import SparkContext
df = sqlContext.createDataFrame(output)

In [33]:
df.show()

In [34]:
df.select('prediction').show(10)

In [35]:
#check and remove the output from the previous run
display(dbutils.fs.ls("/FileStore/model/mycsv.csv"))

In [36]:
%fs rm -r /FileStore/model/mycsv.csv

In [37]:
df.select('prediction').coalesce(1).write.format('com.databricks.spark.csv').save('/FileStore/model/mycsv.csv')

In [38]:
display(dbutils.fs.ls("/FileStore/model/mycsv.csv"))

In [39]:
#%fs head FileStore/model/mycsv.csv/part-00000-tid-1337084786159374092-3f1e8920-b18f-4b39-8b2a-03af2d2e2b3b-7400-c000.csv